In [1]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
import preliz as pz
from scipy.special import expit as logistic
import xarray as xr

In [2]:
az.style.use('arviz-doc')

In [3]:
%%HTML
<style>

.CodeMirror {
    width: 100vw;
}

.container {
    width: 99% !important;
}

.rendered_html {
  font-size:0.8em;
}
.rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
     font-size: 100%;
}


body {
  font-family: Ubuntu;
  background: #F0F0F0;
  background-color: #F0F0F0;
}


.reveal h1,
.reveal h2,
.reveal h3,
.reveal h4,
.reveal h5,
.reveal h6 {
  margin: 0 0 20px 0;
  color: #2a2eec;
  font-family: Ubuntu;
  line-height: 0.9em;
  letter-spacing: 0.02em;
  text-transform: none;
  text-shadow: none;
}

.reveal blockquote {
  display: block;
  position: relative;
  background: #fa7c17;
  border-radius: 15px;
  box-shadow: 0px 0px 2px rgba(0, 0, 0, 0.2);
  font-weight: bold;
}

</style>

In [4]:
from traitlets.config.manager import BaseJSONConfigManager
path = "/home/osvaldo/anaconda3/etc/jupyter/nbconfig"
cm = BaseJSONConfigManager(config_dir=path)
cm.update("livereveal", {
              "theme": "serif",
              "transition": "zoom",
              "start_slideshow_at": "selected",
              "controls": "True",
              "progress": "False",
              "shortcut": "False"});

<center><img src="img/Logo_UNSAM.png" width="200"></center>
<br>
<br>
<h1 align="center">Árboles de regresión aditivos Bayesianos</h1>    


<br>
<br>
<br>
<br>
<br>

## Objetivos

<br>

* Árboles de decisión
* Modelos BART
* Regresión flexible con BART
* Gráficos de dependencia parcial
* Gráficos de expectativas condicionales individuales
* Selección de variables

## Funciones no-lineales

<br>
Vimos que podíamos excribir GLMs como:

$$
\begin{aligned}
\mu &= \alpha + \beta X \\
Y &\sim \phi(L(\mu), \theta)
\end{aligned}
$$

* El objetivo de este capítulo es generar modelos de la forma de:

$$
\begin{aligned}
\mu &= f(X) \\
Y &\sim \phi(L(\mu), \theta)
\end{aligned}
$$

donde $f$ es una función que aproximamos de forma no paramétrica.
<br>


## Árboles vs GPs

<br>

* Anteriormente vimos que era posible modelar funciones usando GPs como priors
* Ahora vamos a ver una alternativa basada en árboles

<br>

## Árboles al rescate

<br>


* Informalmente podemos pensar un árbol como un diagrama de flujo
* Empezamos siempre desde la "raíz" y vamos contestando una serie de pregunta del tipo si/no
* Eventualmente alcanzaremos una "hoja" que tendrá la respuesta a nuestro problema.

<br>


## Árboles de decisión

<br>
Supongamos que tenemos dos variables $X_1$ y $X_2$ y queremos usar esas variables para clasificar objetos en dos clases: ⬤ o ▲. 

<br>

<center><img src="../img/decision_tree.png" width="700"></center>


## Árboles formales

<br>

* Un poco más formal podemos decir que:
    * Un árbol es un conjunto de nodos y lados, es decir un gráfo.
    * Los árboles NO tiene ciclos, de forma que cualquier par de nodos está conectado por un único camino.
    * El nodo raiz NO tiene padres
    * Los nodos hoja NO tienen hijos 
    * Un árbol binario es aquel en donde cualquier nodo no tiene más de dos hijos 
        * A veces se define como cada nodo no-hoja tiene exactamente dos hijos

<br>

## Árboles de regresión

<br>
<br>

* Cuando los valores de los nodos hojas son clases --> árboles de decisión
* Cuando los valores de los nodos hojas son reales --> árboles de regresión
* En la práctica es común tener combinaciones

<br>
<br>


<center><img src="../img/decision_tree_reg.png" width="700"></center>

## Árboles y splines

<br>
<br>

* Vimos que con los splines podíamos definir una función como una suma pesada de funciones base.
* Para ellos utilizamos un tipo particular de polinomio definidos/restringidos utilizando un conjunto de nudos.
* Si restringuimos los polinomios a orden 0, es decir constantes, los splines y los árboles son equivalentes.
<br>
<br>

![](../img/splines_weighted.png)

## Árboles y splines

<br>
<br>

* La diferencia está en como se usan!
* Para los splines necesitamos definir los nudos de antemano, en los árboles podemos "aprenderlos"
* Por construcción los splines suelen estar limitados a pocas dimensiones (1, 2, 3, ...), los árboles no.

<br>
<br>

## Interpretabilidad Flexibilidad

* Los árboles, como los que hemos visto, representan funciones escalonadas
* En principio podemos aproximar cualquier función continua con una función escalonada
* Los árboles ofrecen un método muy flexible.
* La contracara de esta flexibilidad es el sobreajuste. La siguiente figura muestra un ejemplo de un árbol que ha sobreajustado los datos.   


Una característica atractiva de los árboles de decisión/regresión es su interpretabilidad: literalmente se puede leer el árbol y seguir los pasos necesarios para resolver un determinado problema. Y, por lo tanto, se puede comprender de forma transparente qué está haciendo el método, porque funciona y porque algunas clases pueden no estar clasificadas adecuadamente o porque algunos datos no están bien aproximados. Además, también es fácil explicar el resultado a una audiencia no técnica con términos simples.

Además los árboles ofrecen un método muy flexible, ya que siempre se puede encontrar un árbol lo suficientemente complejo como para que cada nodo hoja tenga asociado una observación. La contracara de esta flexibilidad es el sobreajuste. La siguiente figura muestra un ejemplo de un árbol que ha sobreajustado los datos.   


## Interpretabilidad y flexibilidad

<br>
<br>

* Una característica atractiva de los árboles es su interpretabilidad: literalmente se puede leer el árbol y seguir los pasos necesarios para resolver un determinado problema
* Los árboles pueden representar tanto efectos principales como interacciones. 
    * Cuando un árbol depende de una sola variable $X_0$, estamos representando un efecto principal
    * Cuando un árbol depende de más de una variable $X_0, X_1, \dots$ estamos representando una interacción. 
* Como la profundidad de los árboles es arbitraria, en principio podemos modelar interaccioens de orden arbitrario*
* En la práctica estimar una interacción requiere de más datos que estimar una efecto principal y estimar una interacción de bajo orden requiere menos datos que una de mayor orden.

<br>

## Interpretabilidad y flexibilidad

<br>
<br>

* Los árboles son muy flexible, en principio es posible encontrar un árbol lo suficientemente complejo como para ajustar cualquier dataset.
* La contracara es el sobreajuste. 

<br>
<br>

![](../img/decision_tree_overfitting.png)

## Regularizando los árboles

<br>
<br>

* Es común introducir dispositivos para regularizar la complejidad de los árboles y disminuir el riesgo de sobreajuste. 
* Una solución común es utilizar un conjunto de árboles y no un solo árbol
* Cada árbol está restringido en uno o más aspectos, por ej la profundidad, de esa forma no puede sobreajustar
* Luego los árboles se combinan para genera una única respuesta.

<br>
<br>

## Regularizando los árboles

<br>
<br>

* Estrictamente una suma de árboles es equivalente a un nuevo árbol más profundo.
* En principio sería posible trabajar con un solo árbol y encontrar una solución que no sobreajuste. Pero empiricamente esta tarea no es simple
* Una desventaja de utilizar conjuntos de árboles es que perdemos la interpretabilidad de un único árbol.
* Para obtener una respuesta no podemos seguir un solo árbol, debemos considerar a todos, lo que generalmente dificulta cualquier interpretación directa.
* Hemos cambiado la interpretabilidad por la flexibilidad y la generalización

<br>
<br>

## BART


De forma general podemos escribir un modelo BART como:

$$
Y = \phi \left(\sum_{j=0}^m G(\boldsymbol{X}; \mathcal{T}_i, \mathcal{M}_i), \theta \right)
$$

* $G$ representa una función árbol parametrizada por 
    * $\mathcal{T}_i$, la estructucta del grafo junto con las reglas de decisión asociadas con los nodos internos
    * $\mathcal{M}_i$, el conjunto de valores de los nodos hojas.
* $\phi$ representa una distribución de probabilidad arbitraria que se usará como likelihood en nuestro modelo
* $\theta$ otros parámetros de $\phi$ no modelados como una suma de árboles.

## Priors para BART


* En el [artículo original](https://arxiv.org/abs/0806.3286) de BART y en general en posteriores modificaciones, los priors para BART son conjugados. En PyMC-BART no
* Para simplificar la especificación de los priors asumimos que la estructura del árbol $\mathcal{T}$ y los valores de las hojas $\mathcal{M}$ son independientes.
* Además estos priors son independientes de los priors para $\theta$. 


## Priors para BART

<br>
<br>

El prior para $\mathcal{T}$ se especifica como:

* La probabilidad de que un nodo de profundidad $d=(0, 1, 2, \dots)$ sea interno -->  $\alpha(1 + d)^{-\beta}$ con $\alpha \in (0, 1)$ y $\beta \in [0, \infty)$.
* La distribución sobre la variable de partición. Uniforme entre las covariables disponibles --> PyMC-BART lo ajusta durante la fase de tuning
* La distribución sobre los valores de partición. Uniforme sobre los valores disponibles.

<br>
<br>

## Priors para BART

<br>
<br>


El prior para los valores de las hojas $\mu_{ij}$


* Usamos $\mathcal{N}(\mu_\text{pred}, {\varepsilon^2})$, donde $\mu_\text{pred}$ se calcula como la media de la suma actual de árboles dividida por número de árboles $m$. 
* El valor inicial de $\varepsilon$ se calcula a partir de $Y$, siendo $\varepsilon = \frac{3}{\sqrt{m}}$ para datos binomiales y $\varepsilon = \frac{Y_\text{std}}{\sqrt{m}}$ para datos distintos del binomial, pero durante la fase de tuning se ajusta



<br>
<br>

## Priors para BART

<br>
<br>


* El número de árboles $m$ debe ser especificado, los valores comunmente usados suelen ser 50, 100 o 200.
* En principio se podría colocar un prior sobre $m$ pero en la práctica parece que nadie a encontrado una forma eficiente/correcta de hacerlo.
* Es posible usar validación cruzada para determinar $m$, también es posible usar LOO.

<br>
<br>

## Inferencia sobre árboles de regresión aditiva bayesiana

<br>

* Hasta ahora hemso discutido como especificar árboles, pero no como hacer inferencia.
* Para ajustar árboles no podemos usar métodos basados en gradientes como HMC

A grandes rasgos, cada paso del método implementado en PyMC-BART consiste en elegir uno de los $m_i$ árboles disponibles y proponer un nuevo árbol que lo reemplace. Para ello se procede de la siguiente forma:

1) Se hacen crecer $N$ árboles, comenzando desde la raíz y siguiendo los priors.
2) Se calcula un peso para cada uno de los $N$ árboles y para el árbol $m_i$
3) Se reemplaza el árbol $m_i$ por un árbol muestreado de forma proporcional a los pesos del punto 2

<br>

[Detalles](https://arxiv.org/abs/2206.03619)

## Inferencia sobre árboles de regresión aditiva bayesiana

<br>

* En este algoritmo los priors son utilizado como distribución de propuesta, esto no es lo más común.
* El peso calculado en el punto 2 es el log-likelihood, teniendo en cuenta la suma del árbol de propuesta y todos los demás árboles $m_{-i}$. 
* Siempre es posible elegir el árbol que se intenta reemplazar. Es decir es posible "quedarse en el lugar"
* Como sucede con métodos MCMC, la probabilidad de elegir un árbol que "empeore" el ajuste es no nula.

<br>

## Minería de carbón con BART

* Vamos a utilizar el mismo ejemplo que usamos con GPs

In [ ]:
# discretizar datos
years = int(coal.max() - coal.min())
bins = years // 4
hist, x_edges = np.histogram(coal, bins=bins)
# Calcular la ubicación de los centros de los datos discretizados
x_centers = x_edges[:-1] + (x_edges[1] - x_edges[0]) / 2
# x_data debe ser 2D para BART
x_data = x_centers[:, None]
# Expresar los datos como la tasa de número de accidentes por año
y_data = hist

In [ ]:
with pm.Model() as modelo_coal:
    μ_ = pmb.BART("μ_", X=x_data, Y=np.log(y_data), m=20)
    μ = pm.Deterministic("μ", pm.math.exp(μ_))
    y_pred = pm.Poisson("y_pred", mu=μ, observed=y_data)
    idata_coal = pm.sample(random_seed=RANDOM_SEED)